MuxConv 알고리즘 완성, FHE 적용 전에 Pure Python으로 테스트하는 스크립트. 
Pure Python버전 정상 작동 됨.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from time import time
from muxcnn.utils import load_params, load_img
import torch
from muxcnn.resnet_muxconv import ResNet_MuxConv

## Torch model

In [3]:
from muxcnn.models.ResNet20 import ResNet, BasicBlock

model = ResNet(BasicBlock,[1,1,1])
model.eval() ########## 필수! 
load_params(model, fn_param="./ResNet8.pt",device='cpu')

In [4]:
img_tensor = load_img("./cute.jpg", hi=32, wi=32)

In [5]:
print(model(img_tensor))

before softmax tensor([[ -9.3005, -11.0586,  -2.9741,  -2.9179,  -9.3500,   4.1474,  -6.8721,
          -3.6999,  -8.9446,  -7.6901]], grad_fn=<AddmmBackward0>)
tensor([[-1.3450e+01, -1.5208e+01, -7.1236e+00, -7.0674e+00, -1.3500e+01,
         -2.0791e-03, -1.1022e+01, -7.8494e+00, -1.3094e+01, -1.1840e+01]],
       grad_fn=<LogSoftmaxBackward0>)


# FHE


ResNet_MuxConv(model, alpha=12) 와 결과 비교

`scripts/MuxConv_parBN_final_example.ipynb`

In [6]:
from muxcnn.hecnn_par import *

In [7]:
imgl = get_channel_last(img_tensor[0].detach().numpy())
ki = 1
hi, wi, ch = imgl.shape

U, ins, outs = get_conv_params(model.conv1, {'k':ki, 'h':hi, 'w':wi})
ct_a = MultParPack(imgl, ins)
out, un1 = forward_conv_par(model.conv1, ct_a, ins)

U, ins, outs = get_conv_params(model.layer1[0].conv1, outs)
out, un2 = forward_conv_par(model.layer1[0].conv1, out,  ins)

U, ins, outs = get_conv_params(model.layer1[0].conv2, outs)
out, un3 = forward_conv_par(model.layer1[0].conv2, out,  ins)

U, ins, outs = get_conv_params(model.layer2[0].conv1, outs)
out, un4 = forward_conv_par(model.layer2[0].conv1, out, ins)

U, ins, outs = get_conv_params(model.layer2[0].conv2, outs)
out, un5 = forward_conv_par(model.layer2[0].conv2, out, ins)

U, ins, outs = get_conv_params(model.layer3[0].conv1, outs)
out, un6 = forward_conv_par(model.layer3[0].conv1, out, ins)

U, ins, outs = get_conv_params(model.layer3[0].conv2, outs)
out, un7 = forward_conv_par(model.layer3[0].conv2, out, ins)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 2
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 8
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 8
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(16,16,32, 2, 8,  4)
[MultParConv] q = 16
[MultParConv] (hi,wi,ci,ki,ti,pi) =(16,16,32, 2, 8,  4)
[MultParConv] (ho,wo,co,ko,to,po) =(16,16,32, 2, 8,  4)
[MultParConv] q = 8
[MultParConv] (hi,wi,ci,ki,ti,pi) =(16,16,32, 2, 8,  4)
[MultParConv] (ho,wo,co,ko,to,po) =( 8, 8,64, 4, 4,  8)
[MultParConv] q = 16
[MultParConv] (hi,wi,ci,ki,ti,pi) =( 8, 8,64, 4, 4,  8)
[MultParConv] (ho,wo,co,ko,to,po) =( 8, 8,64, 4, 4,  8)
[MultParConv] q = 8


In [8]:
out

array([-27.60442039,  43.06821207,  25.64662879, ...,  27.13122852,
        -3.49895375, -26.09628137])

In [9]:
fhe_model = ResNet_MuxConv(model, alpha=12)

functions set
degrees = [15, 15, 15, 15], margin = 0.01, eps = 0.02


In [10]:
t0 = time()
ctxt, outs0 = fhe_model.forward_early(img_tensor)
print(f"{time() - t0:.4f} s")

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
1.2221 s
1.2304 s


In [11]:
ctxt

array([ 1.23083757e+00,  4.86967574e-01,  2.75344729e-01, ...,
        7.15849049e-11,  6.76558445e-11, -1.68027319e-11])

In [8]:
print(ctxt[::1000])

[ 1.23083757e+00 -6.04646501e-12  8.31757005e-01  5.57055145e-01
  1.46344638e+00  5.52875764e-01  2.28876953e-11  3.22529486e-01
  1.66880482e-01  1.38492884e-11 -4.83460327e-12  5.25957646e-02
 -2.84754336e-10  8.14742654e-01  3.52006406e-01  9.86855253e-01
 -1.37065966e-10  2.00778844e-12  2.00753381e-01  4.62600852e+00
  1.45841416e+00  1.12151311e-01  2.53264000e-01  2.61802319e+00
  6.75758608e-02  2.28868992e-02 -1.00897592e-10  1.05486982e-10
  2.94736493e-10  9.83460246e-01  9.61286444e-01  4.02652655e-01
  1.21398574e-10]


In [9]:
ctxt2, outs1 = fhe_model.forward_bb(model.layer1[0], ctxt, outs0)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
4.7524 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
4.7195 s


In [11]:
print(ctxt2[::1000])

[ 2.29267628e+00  1.55754488e-01  2.07210187e-01  6.34624115e-01
  1.73381882e+00  4.63571208e-11  9.20636667e-01  1.24996979e-01
  7.27311266e-01  1.06821348e-10  4.58710870e-01  9.80976026e-01
 -6.73565785e-11  1.77517230e+00  1.86435264e-01  8.93211277e-01
  9.10426754e-01  4.98329769e-04  1.02342104e+00  1.03988114e+00
  1.41273578e+00  2.18611558e-11  1.30317306e-01  1.31060163e+00
  1.66335217e-01  1.28632700e-10 -1.37378555e-10  1.61787374e-10
 -2.51920277e-12  7.86957636e-01  1.64642152e+00 -3.03935162e-12
  8.46394818e-01]


In [12]:
ctxt2, outs1 = fhe_model.forward_bb(model.layer1[0], ctxt2, outs1)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
4.6834 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
4.7635 s


/home/hoseung/anaconda3/envs/fhe/lib/python3.10/site-packages/numpy/polynomial/polynomial.py:756: RuntimeWarning: overflow encountered in multiply
  c0 = c[-i] + c0*x
/home/hoseung/anaconda3/envs/fhe/lib/python3.10/site-packages/numpy/polynomial/polynomial.py:754: RuntimeWarning: invalid value encountered in multiply
  c0 = c[-1] + x*0


In [14]:
t0 = time()
result = fhe_model(img_tensor)
print(f"{time() - t0:.4f} s")

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
1.2153 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
4.7146 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
4.7215 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
9.1421 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(16,16,32, 2, 8,  4)
4.3857 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
1.3069 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(16,16,32, 2, 8,  4)
8.2088 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =( 8, 8,64, 4, 4,  8)
4.0390 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(16,16,32, 2, 8,  4)
1.2497 s
39.0005 s


In [15]:
torch.tensor(result[::64][:10])

tensor([ -9.2678, -10.8151,  -3.1035,  -2.9026,  -9.4615,   4.3280,  -6.9753,
         -3.5617,  -8.8841,  -7.5631], dtype=torch.float64)